# Modelling I – fitting logistic regression on HomeCredit data

This notebook contains an example of fitting and evaluating logistic regression model for the lecture Modelling I.

## Data

We use the basic dataset of clients *application_train.csv*.

* target (default = did not repay the loan)
* loan type
* sex, age, family status, count of children and family members
* ownership of car, realty, income type
* 3 external scores

In [ ]:
# setup
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pandas as pd
import numpy as np
import seaborn as sns

pd.set_option("display.precision", 2)

In [ ]:
# Home Credit data reading and preparing (a little)
df_hc = pd.read_csv('application_train.csv') # adjust file path
df_hc.columns = df_hc.columns.str.lower()

# data reduction - selection of columns
df_hc_colnames = ['sk_id_curr', 'target', 'name_contract_type', 'code_gender', 'flag_own_car',
                 'flag_own_realty', 'name_income_type', 'days_birth', 'ext_source_1',
                 'ext_source_2', 'ext_source_3', 'cnt_children', 'cnt_fam_members', 'name_family_status']
df_hc = df_hc[df_hc_colnames]
# and renaming to simpler names
df_hc_colnames_new = ['id', 'target', 'loan_type', 'sex', 'has_car',
                 'has_realty', 'income_type', 'age_days', 'score1',
                 'score2', 'score3', 'cnt_children', 'cnt_fam_members', 'fam_status']
df_hc.columns = df_hc_colnames_new

# data transformation
df_hc['age_days'] = -df_hc['age_days']
df_hc['age'] = df_hc['age_days'] / 365.25

## Logistic regression
We take *target* as target and *score2* (and later *sex* and *age*) as predictor(s).

In [ ]:
# data preparation
df_hc = df_hc[df_hc['target'].notna() & df_hc['score2'].notna() & df_hc['sex'].notna() & df_hc['age'].notna()]
df_hc = df_hc[df_hc['sex']!='XNA']

In [ ]:
print('Target positive rate: ', np.mean(df_hc['target']))

In [ ]:
# chart of score2
g = sns.displot(data=df_hc, x="score2") \
    .set_axis_labels("Score2", "Density") \
    .set(title="Distribution of Score2")

In [ ]:
### let's go modelling
X = df_hc[['score2']]
y = df_hc['target']

# fit model
modelA = LogisticRegression(solver='newton-cg', penalty='none').fit(X, y)

# get coefficients
print('Intercept: ', modelA.intercept_)
print('Beta coefficients: ', modelA.coef_)

In [ ]:
# chart of fitted positive rate vs. score
hlp_x = np.array(range(101)) / 100.0 # fitting for score from 0 to 1
hlp_x2 = [[x] for x in hlp_x.tolist()]
hlp_y = modelA.predict_proba(hlp_x2) # fitted probabilities
hlp_y = [x[1] for x in hlp_y] # transformation to a list
hlp_x2 = [x[0] for x in hlp_x2] # unlisting the list
hlp_df = pd.DataFrame({'score2': hlp_x2, 'pred': hlp_y}) # auxilliary DataFrame for seaborn

g = sns.relplot(data=hlp_df, x="score2", y="pred") \
    .set_axis_labels("Score2", "Probability") \
    .set(title="Predicted probability")

After fitting the model, let's evaluate the performance.

We can compare fitted vs. actual target values on all records, but this is not recommended -- it overrates the model. The correct way is to do **cross-validation**.

The **metric** use by default in sklearn LogisticRegression is **accuracy**. It's not good in unbalanced situations like here:
* it uses prediction threshold 0.5 when we work with small (or big) probabilities;
* predicting all negative or all positive gives high accuracy

In [ ]:
### now evaluate model
# assess model performance
# i. scoring itself directly
print('Accuracy on itself: ', modelA.score(X, y))
# Bad for two reasons:
# 1. generally overestimates the performance
# 2. if the target is highly unbalanced, accuracy is bad matric

# ii. scoring by a cross-validation
# but use smarter metric instead of accuracy - e. g. ROC AUC
scores = cross_val_score(LogisticRegression(solver='newton-cg', penalty='none'), X, y, cv=5,
                         scoring='roc_auc')
print('ROC AUC by cval: ', scores)

It looks like *score2* makes our model much better than null model. The ROC AUC is about 0.65 (null model gives 0.5).

We can assure ourselves in the significance of *score2* predictor by other view provided by `statmodels` package.

In [ ]:
modelC = smf.logit("target ~ score2", data=df_hc).fit()
print(modelC.summary()) # detailed information of model and coefficients

> The coefficient at *score2* has p-value much lower than 0.05 threshold, therefore it is significantly different from zero and statistically significant.

Now we add two predictors more: *sex* and *age*.

In [ ]:
# data preparation
# it's necessary to make dummies for categorical predictor "sex"
df_hc['sex_orig'] = df_hc['sex'] # backup of the original column - it will be lost!
df_hc = pd.get_dummies(df_hc, columns=['sex'], drop_first=True)
df_hc.rename(columns = {'sex_orig':'sex'}, inplace = True)

In [ ]:
X = df_hc[['score2', 'age', 'sex_M']]
y = df_hc['target']

# fit model
modelA = LogisticRegression(solver='newton-cg', penalty='none').fit(X, y)

# get coefficients
print('Intercept: ', modelA.intercept_)
print('Beta coefficients: ', modelA.coef_)

In [ ]:
scores = cross_val_score(LogisticRegression(solver='newton-cg', penalty='none'), X, y, cv=5,
                         scoring='roc_auc')
print('ROC AUC by cval: ', scores)

As we can see, the model performance has slightly increased (0.65 -> 0.67). Is the model with three predictors significantly better than the former model? It seems so, but we can judge it by p-values at coefficients for *age* and *sex*. 

In [ ]:
modelC = smf.logit("target ~ score2 + age + sex_M", data=df_hc).fit()
print(modelC.summary()) # detailed information of model and coefficients

> Again, all coefficients at predictors are statistically significant -- different from zero.

We can compare predicted and actual values. It should be made on predictions from cross-validation, but just for illustration, let's do it on predictions from model itself.

In [ ]:
X_pred = df_hc.sample(50)[['target', 'score2', 'age', 'sex_M']]
y_pred = modelC.predict(X_pred) # fitted probabilities
y_act = X_pred[['target']] # actual target values
y_act['pred'] = y_pred
y_act

As we can see, all predicted probabilites are below 0.5. So for applying any smart metric, one have to put the threshold lower than 0.5. Let's try threshold 0.15 and show the confusion matrix.

In [ ]:
from sklearn.metrics import confusion_matrix
y_class = (y_pred > 0.15)
confusion_matrix(y_act['target'], y_class)

## Try it yourself
Now try to use another columns as predictors and to improve risk model. Consider binning some numeric columns to categories.